# A Simple Problem

Let us consider a simple problem of sizing and planning a small energy system made-up of a solar PV plant and a battery. The goal is to find the optimal capacity of PV panels and battery to install in order to answer a certain known electricity load and also plan when to charge and discharge the battery.

In [ ]:
my_problem = """
#TIMEHORIZON 
    T = 24; // planning horizon (hours)

#GLOBAL
    wacc = 0.07;
    number_years_horizon = T/8760;

#NODE SOLAR_PV_PLANTS
    #PARAMETERS
        full_capex = 380.0;
        lifetime = 25.0;
        annualised_capex = full_capex * global.wacc * (1 + global.wacc)**lifetime / ((1 + global.wacc)**lifetime - 1); // MEur
        fom = 7.25; // MEur/year
        vom = 0.0;
        capacity_factor_PV = import "../remote_energy_supply_chain/pv_capacity_factors.csv"; // Dimensionless
        max_capacity = 500.0; // GW
    #VARIABLES
        internal: capacity;
        external: electricity[T];
    #CONSTRAINTS
        electricity[t] <= capacity_factor_PV[t] * capacity;
        capacity <= max_capacity;
        capacity >= 0;
        electricity[t] >= 0;
    #OBJECTIVES
        min: global.number_years_horizon * (annualised_capex + fom) * capacity;
        min: vom * electricity[t];
        

#NODE BATTERY_STORAGE
    #PARAMETERS
        full_capex_stock = 142.0;
        full_capex_flow = 160.0;
        lifetime_stock = 10.0;
        lifetime_flow = 10.0;
        annualised_capex_stock = full_capex_stock * global.wacc * (1 + global.wacc)**lifetime_stock / ((1 + global.wacc)**lifetime_stock - 1); // MEur
        annualised_capex_flow = full_capex_flow * global.wacc * (1 + global.wacc)**lifetime_flow / ((1 + global.wacc)**lifetime_flow - 1); // MEur
        fom_stock = 0.0;
        fom_flow = 0.5;
        vom_stock = 0.0018;
        vom_flow = 0.0;
        charge_discharge_ratio = 1.0;
        self_discharge = 0.00004;
        efficiency_in = 0.959;
        efficiency_out = 0.959;
    #VARIABLES
        internal: capacity_flow;
        internal: capacity_stock;
        internal: electricity_stored[T];
        external: electricity_in[T];
        external: electricity_out[T];
    #CONSTRAINTS
        electricity_in[t] <= capacity_flow;
        electricity_out[t] <= charge_discharge_ratio * capacity_flow;
        electricity_stored[t] <= capacity_stock;
        electricity_stored[0] == electricity_stored[T-1];
        electricity_stored[t+1] == (1 - self_discharge) * electricity_stored[t] + efficiency_in * electricity_in[t] - electricity_out[t] / efficiency_out;
        capacity_flow >= 0;
        capacity_stock >= 0;
        electricity_stored[t] >= 0;
        electricity_in[t] >= 0;
        electricity_out[t] >= 0;
    #OBJECTIVES
        min: global.number_years_horizon * (annualised_capex_stock + fom_stock) * capacity_stock + global.number_years_horizon * (annualised_capex_flow + fom_flow) * capacity_flow;
        min: vom_stock * electricity_stored[t] + vom_flow * electricity_in[t];

#HYPEREDGE POWER_BALANCE
    #PARAMETERS
        electrical_load = import "../microgrid/demand.csv";
    #CONSTRAINTS
        SOLAR_PV_PLANTS.electricity[t]+BATTERY_STORAGE.electricity_out[t] == electrical_load[mod(t,24)] + BATTERY_STORAGE.electricity_in[t];
"""

# We write the problem in a file
with open("toy_problem.txt", "w") as f:
  f.write(my_problem)

# How to run the model
This model is saved in the file named "toy_problem.txt". In order to run this model, the following command can be used: 
> gboml toy_problem.txt --gurobi --json --output toy_problem

gboml calls the GBOML compiler upon the preceeding file.
toy_problem.txt is the name of the file
--gurobi tells the compiler to set the option to the gurobi optimizer
--json tells the compiler to output the solution as a JSON file
--output renames the output to the name that comes after, in this case a file named toy_problem.json will be produced


In [ ]:
! gboml toy_problem.txt --gurobi --json --output toy_problem

# Show solution
The command "cat" is used to show a file in the terminal directly. 

In [ ]:
! cat toy_problem.json

In [ ]:
import json

with open("toy_problem.json") as json_file:
    toy_problem = json.load(json_file)
    json_file.close()

toy_problem["solution"]

# Adding a component 
We want to add wind turbines to our simplistic model. In order to do that, first let's have a look at a more sophisticated example, the remote renewable energy hub.  

## Remote renewable energy hub

First let us show the file in itself and run it to check that everything works. We will also check that there is a wind turbines node in it. Again, we use the command "cat" as mentionned above. 

In [ ]:
! cat ../remote_energy_supply_chain/remote_hub_wacc.txt

Let's run the file in two ways:
- using the gboml python library by importing gboml in a python file
- using the terminal gboml executable as before

In [ ]:
# GBOML python library
from gboml import GbomlGraph

T = 365
gboml_model = GbomlGraph(T)
gboml_model.add_global_parameter("wacc", 0.07)
gboml_model.add_global_parameter("number_years_horizon", T/8760)
nodes, edges = gboml_model.import_all_nodes_and_edges("../remote_energy_supply_chain/remote_hub_wacc.txt")
gboml_model.add_nodes_in_model(*nodes)
gboml_model.add_hyperedges_in_model(*edges)
gboml_model.build_model()
solution, objective, status, solver_info, _, _ = gboml_model.solve_gurobi()
solution_dict = gboml_model.turn_solution_to_dictionary(solver_info, status, solution, objective)
print(solution_dict)

In [ ]:
!gboml ../remote_energy_supply_chain/remote_hub_wacc.txt --gurobi

## Adding WIND_PLANTS
In the above mentionned remote renable hub, we found a node named WIND_PLANTS that models wind turbines that we wish to add to our small toy problems. 

Here again, we can proceed in two ways:
- via the GBOML file 
- via the gboml python library 

In the GBOML file, we can import the WIND_PLANTS by writing 
> #NODE WIND_PLANTS = import WIND_PLANTS from "../remote_energy_supply_chain/remote_hub_wacc.txt" with lifetime = 35.0;

We say that we import the node named "WIND_PLANTS" from the file "remote_hub_wacc.txt" and change one of its parameters named "lifetime" by the value 35.0. Furthermore, we change the graph topology by adding the electricty of the wind farm in the model. 

The file is therefore written as follows and saved in "toy_problem2.txt", 

In [ ]:
my_problem = """
#TIMEHORIZON 
    T = 24; // planning horizon (hours)

#GLOBAL
    wacc = 0.07;
    number_years_horizon = T/8760;

#NODE SOLAR_PV_PLANTS
    #PARAMETERS
        full_capex = 380.0;
        lifetime = 25.0;
        annualised_capex = full_capex * global.wacc * (1 + global.wacc)**lifetime / ((1 + global.wacc)**lifetime - 1); // MEur
        fom = 7.25; // MEur/year
        vom = 0.0;
        capacity_factor_PV = import "../remote_energy_supply_chain/pv_capacity_factors.csv"; // Dimensionless
        max_capacity = 500.0; // GW
    #VARIABLES
        internal: capacity;
        external: electricity[T];
    #CONSTRAINTS
        electricity[t] <= capacity_factor_PV[t] * capacity;
        capacity <= max_capacity;
        capacity >= 0;
        electricity[t] >= 0;
    #OBJECTIVES
        min: global.number_years_horizon * (annualised_capex + fom) * capacity;
        min: vom * electricity[t];
        

#NODE BATTERY_STORAGE
    #PARAMETERS
        full_capex_stock = 142.0;
        full_capex_flow = 160.0;
        lifetime_stock = 10.0;
        lifetime_flow = 10.0;
        annualised_capex_stock = full_capex_stock * global.wacc * (1 + global.wacc)**lifetime_stock / ((1 + global.wacc)**lifetime_stock - 1); // MEur
        annualised_capex_flow = full_capex_flow * global.wacc * (1 + global.wacc)**lifetime_flow / ((1 + global.wacc)**lifetime_flow - 1); // MEur
        fom_stock = 0.0;
        fom_flow = 0.5;
        vom_stock = 0.0018;
        vom_flow = 0.0;
        charge_discharge_ratio = 1.0;
        self_discharge = 0.00004;
        efficiency_in = 0.959;
        efficiency_out = 0.959;
    #VARIABLES
        internal: capacity_flow;
        internal: capacity_stock;
        internal: electricity_stored[T];
        external: electricity_in[T];
        external: electricity_out[T];
    #CONSTRAINTS
        electricity_in[t] <= capacity_flow;
        electricity_out[t] <= charge_discharge_ratio * capacity_flow;
        electricity_stored[t] <= capacity_stock;
        electricity_stored[0] == electricity_stored[T-1];
        electricity_stored[t+1] == (1 - self_discharge) * electricity_stored[t] + efficiency_in * electricity_in[t] - electricity_out[t] / efficiency_out;
        capacity_flow >= 0;
        capacity_stock >= 0;
        electricity_stored[t] >= 0;
        electricity_in[t] >= 0;
        electricity_out[t] >= 0;
    #OBJECTIVES
        min: global.number_years_horizon * (annualised_capex_stock + fom_stock) * capacity_stock + global.number_years_horizon * (annualised_capex_flow + fom_flow) * capacity_flow;
        min: vom_stock * electricity_stored[t] + vom_flow * electricity_in[t];

#NODE WIND_PLANTS = import WIND_PLANTS from "../remote_energy_supply_chain/remote_hub_wacc.txt" with
        lifetime = 35.0;

#HYPEREDGE POWER_BALANCE
    #PARAMETERS
        electrical_load = import "../microgrid/demand.csv";
    #CONSTRAINTS
        SOLAR_PV_PLANTS.electricity[t]+BATTERY_STORAGE.electricity_out[t]+WIND_PLANTS.electricity[t] 
           == electrical_load[mod(t,24)] + BATTERY_STORAGE.electricity_in[t];
"""

# We write the problem in a file
with open("toy_problem2.txt", "w") as f:
  f.write(my_problem)

We run the model

In [ ]:
! gboml toy_problem2.txt --gurobi --nb_processes 4

We do a similar thing in the python interface.

In [ ]:
from gboml import GbomlGraph

T = 24
gboml_model = GbomlGraph(T)
gboml_model.add_global_parameter("wacc", 0.07)
gboml_model.add_global_parameter("number_years_horizon", T/8760)
nodes_toy, _ = gboml_model.import_all_nodes_and_edges("toy_problem.txt")
_, hyperedges_toy2 = gboml_model.import_all_nodes_and_edges("toy_problem2.txt")
node_wind = gboml_model.import_node("../remote_energy_supply_chain/remote_hub_wacc.txt",
                                    "WIND_PLANTS", copy=True)
gboml_model.add_nodes_in_model(*nodes_toy)
gboml_model.add_nodes_in_model(node_wind)
gboml_model.add_hyperedges_in_model(*hyperedges_toy2)
gboml_model.build_model()
solution = gboml_model.solve_gurobi()
print(solution)

# Multi-processing
In order to use multiprocessing to generate the model, one can simply add the number of processes wanted as follows, 
- In the gboml executable by adding the option "--nb_processes" followed by a number as  
> gboml toy_problem2.txt --gurobi --nb_processes 4
- In the python interface by adding a number to the function "build_matrix" 
> gboml_model.build_model(4)


# Clean-up the temporary files.

In [ ]:
! rm toy_problem.txt toy_problem2.txt toy_problem.json